# Get Started with Flower Framework

Welcome to Federated Learning Tutorial using Flower Framework. Flower is a unified approach to federated learning, analytics, and evaluation. Open source, python, and easy to learn and personalize.

https://flower.ai/

## Step 0: Preparation

Before we begin with any actual code, let's make sure that we have everything we need.

### Instaling dependencies
First, we should install the necessary packages. In this case we are using Flower Framework with the Simulation module.

In [5]:
## Jump to the next block of code if you already installed the packages.

# Linux
!pip install 'cryptography<45,>=44.0.1'
!pip install 'protobuf>=4.21.6,<5.0.0'
!pip install -q flwr[simulation]

# MacOs
#!pip3 install -U 'flwr[simulation]'

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [6]:
import numpy as np

import flwr as fl
from flwr.common import Metrics

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: Runtime > Change runtime type > Hardware accelerator: GPU > Save). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting DEVICE = torch.device("cpu"). If the runtime has GPU acceleration enabled, you should see the output Training on cuda, otherwise it'll say Training on cpu.

### Loading the data

Federated learning can be applied to many different types of tasks across different domains. In this tutorial, we introduce federated learning by training a Logistic Regression on the popular Iris dataset.

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original Iris dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes.

Each organization will act as a client in the federated learning system. So having 3 organizations participate in a federation means having 3 clients connected to the federated learning server.


Let's now create the Federated Dataset abstraction that from flwr-datasets that partitions the Iris. We will create small training and test set for each edge device and wrap each of them into a DataLoader:

In [7]:
NUM_CLIENTS = 3

def load_data():
    data = load_iris()
    X, y = data.data, data.target

    # Filter to remove one class, and force the problem to be binary to run a logistic regression
    mask = y != 2
    X_filtered = X[mask]
    y_filtered = y[mask]

    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def partition_data(X, y, num_clients):
    partition_size = len(X) // num_clients
    partitions = [(X[i * partition_size:(i + 1) * partition_size], y[i * partition_size:(i + 1) * partition_size])
                  for i in range(num_clients)]
    return partitions


## Step 2: Implementing Flower Client
Federated Learning systems consist of a server and multiple clients. In Flower, we create clients by implementing subclasses of `flwr.client.Client` or `flwr.client.NumPyClient`. We use `NumPyClient` in this tutorial because it is easier to implement.
To implement the Flower client, we create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`:
* `get_parameters`: Return the current local model parameters.
* `fit`: Receive model parameters from the server, train the model parameters on the local data, and return the updated model parameters to the server.
* `evaluate`: Receive model parameters from the server, evaluate the model parameters on the local data, and return the evaluation result to the server.

Our clients will use the `scikit-learn` components for model training and evaluation. Let's see a simple Flower Client implementation that brings everything together.

In [12]:
from collections import OrderedDict
from typing import Dict, List, Tuple
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

from flwr.common import NDArrays, Scalar, Status, Code

# in the python file you want to add custom messages to the Flower log
from logging import INFO, DEBUG
from flwr.common.logger import log

class SklearnClient(fl.client.NumPyClient):
    def __init__(self, X_train, y_train, X_test, y_test, cid):
      #super().__init__()
      self.model = LogisticRegression(penalty="l2", max_iter=1000, C=1.0, warm_start=True)
      self.X_train = X_train
      self.y_train = y_train
      self.X_test = X_test
      self.y_test = y_test
      self.cid = cid
      self.set_initial_params()

    def set_initial_params(self):
        """Sets initial parameters as zeros Required since model params are uninitialized
        until model.fit is called.
        But server asks for initial parameters from clients at launch. Refer to
        sklearn.linear_model.LogisticRegression documentation for more information.
        """
        unique_classes = np.unique(self.y_train)
        n_classes = len(unique_classes) # Iris has 3 classes, but we reduced to two classes
        n_features = self.X_train.shape[1] # Number of features in dataset
        self.model.classes_ = np.array([i for i in range(n_classes)])

        self.model.coef_ = np.zeros((n_classes, n_features))
        if self.model.fit_intercept:
            self.model.intercept_ = np.zeros((n_classes,))

    def get_parameters(self, config=None):
      if not hasattr(self.model, "coef_"):
        self.model.fit(self.X_train[:1], self.y_train[:1])

      if self.model.fit_intercept:
        params = [
            self.model.coef_,
            self.model.intercept_
        ]
      else:
        params = [
            self.model.coef_
        ]
      return [param.astype(np.float32) for param in params]

    def set_parameters(self, parameters):
      if self.model.fit_intercept:
        self.model.coef_ = parameters[0]
        self.model.intercept_ = parameters[1]
      else:
        self.model.coef_ = parameters[0]
      return self.model

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.fit(self.X_train, self.y_train)
        return self.get_parameters(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        y_pred = self.model.predict(self.X_test)
        loss = 1 - accuracy_score(self.y_test, y_pred)
        accuracy = self.model.score(self.X_test, self.y_test)
        cid_ = self.cid

        report = classification_report(self.y_test, y_pred)
        # print("******* accuracy ")
        # print(accuracy)
        print("*** Classification Report ***:")
        print(classification_report(self.y_test, y_pred))

        #message = "Client " + str(cid_) + " accuracy: " + str(accuracy)
        #log(DEBUG, message)

        return loss, len(self.X_test), {"accuracy": accuracy}


Our class `SklearnClient` defines how local training/evaluation will be performed and allows Flower to call the local training/evaluation through `fit` and `evaluate`. Each instance of `SklearnClient` represents a *single client* in our federated learning system. Federated Learning systems have multiple clients (otherwise, there is not federated), so each client will be represent by its own instance of `SklearnClient`. If we have, for example, three clients in our workload, then we'd have three instances of `SklearnClient`. Flower calls `SklearnClient.fit` on the respective instance when the server selects a particular client for training and `SklearnClient.evaluate` for evaluation.

### Using the Virtual Client Engine

In this notebook, we are simulating a federated learning system with 3 clients on a single machine. This means that the server and all 3 clients will live on a single machine and share resources as CPU, GPU, and memory. Having 3 clients would mean having 3 instances of `SklearnClient` in memory. Doing this on a single machine can quickly exhaust the available memory resources, even if only a subset of this clients participates in a single round of federated learning.

In addition to the regular capabilities where server and clients run on multiple machines, Flower, therefore provides  simulation capabilities that create `SklearnClient` instances only when they are actually necessary for training or evaluation. To enable clients the Flower framework to create clients when necessary, we need to implement a function called `client_fn` that creates a `SklearnClient` instance on demand. Flower calls `client_fn` whenever it needs an instance of one particular client to call `fit` or `evaluate` (those instances are usually discarded after use, so they should not keep any local state). Clients are identified by a client ID, or short `cid`. The `cid` can be clients, as can be seen below.

In [13]:
def client_fn(cid: str) -> fl.client.Client:
    X_train, X_test, y_train, y_test = load_data() # Load data
    num_clients = NUM_CLIENTS  # Number of clients should match the number of partitions
    partitions = partition_data(X_train, y_train, num_clients) # Create data partitions

    partition_id = int(cid) # Associate the partition id to client id (cid)
    # Each client gets a different X_train and y_train, so each client will train and test on their unique data
    X_train_cid, y_train_cid = partitions[partition_id]

    # Create a single Flower client representing a single organization/device
    return SklearnClient(X_train_cid, y_train_cid, X_test, y_test, cid).to_client() #for Flower version > 1.8
    #return SklearnClient(X_train_cid, y_train_cid, X_test, y_test, cid) #for Flower version < 1.8

## Step 3: Define the Strategy

The **strategy** encapsulates the aggregation algorithm, such as *Federated Average (FedAvg)*.
Flower has a number of built-in strategies, but we can also use our own strategy implementations to customize nearly all aspects of the federated learning approach. For this example, we use the buiklt-in *FedAvg* implementation and customize a few basic parameters. The last step is the actual call `start_simulation` which starts the simulation.

In [14]:
def fit_config(server_round) -> Dict:
    """Send round number to client."""
    config = {
        "server_round": server_round
    }
    return config

# Define the strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=1,
    on_fit_config_fn=fit_config
)

# Simulation configuration
# Check the Flower Framework documentation for more details about Flower Simulations
# and how to setup the client_resources
client_resources = {"num_cpus": 1}
num_clients = NUM_CLIENTS
num_rounds = 5

## Step 4: Run Simulation

We now have the class `SklearnClient` which defines client side training/evaluation and `client_fn` which allows Flower to create `SklearnClient` instances whenever it needs to call `fit` or `evaluate` on one particular client. The last step is to start the actual simulation using `flwr.simulation.start_simulation`.
The function `start_simulation` accepts a number of arguments, amongst the `client_fn` used to create `SklearnClient` instances, the number of clients to simulate (`num_clients`), the number of federated learning rounds (`num_rounds`), and the **strategy**.


In [15]:
fl.simulation.start_simulation(
 # strategy=strategy, # the strategy that will construct a client
  client_fn=client_fn, # a function to construct a client
  num_clients=num_clients, # total number of clients in the experiment
  config=fl.server.ServerConfig(num_rounds=5), #let's run for 5 rounds
  client_resources=client_resources,
)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-03-28 11:20:34,134	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7995521435.0, 'object_store_memory': 3997760716.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

(ClientAppActor pid=6479) *** Classification Report ***:
(ClientAppActor pid=6479)               precision    recall  f1-score   support
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6479)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)     accuracy                           1.00        20
(ClientAppActor pid=6479)    macro avg       1.00      1.00      1.00        20
(ClientAppActor pid=6479) weighted avg       1.00      1.00      1.00        20
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479) *** Classification Report ***:
(ClientAppActor pid=6479)               precision    recall  f1-score   support
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6479)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)     a

(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO

(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6479)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479) 
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6480)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480) 
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6479)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479) 
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6480)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480) 
(ClientAppActor pid=6479) 
(ClientAp

(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
(ClientAppActor pid=6479) 
(ClientAppActor pid=6479)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 0.84s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      		round 3: 0.0
INFO :      		round 4: 0.0
INFO :      		round 5: 0.0
INFO :      


(ClientAppActor pid=6480) 
(ClientAppActor pid=6480)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6480)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6480) 
(ClientAppActor pid=6480) 


History (loss, distributed):
	round 1: 0.0
	round 2: 0.0
	round 3: 0.0
	round 4: 0.0
	round 5: 0.0

So how does this work? How does Flower execute this simulation?
When we call `start_simulation`, we tell Flower that there are 3 clients (`num_clients = 3`). Flower then goes ahead an asks the `FedAvg` strategy to select clients. `FedAvg` knows that it should select 100% of the available clients (`fraction_fit = 1.0`), so it goes ahead and selects 3 randon clients (i.e., 100% of 3).
Flower then asks the selected 3 clients to train the model. When the server receives the model parameter updates from the clients, it hands those updates over the *strategy* (FedAvg) for aggregation. The *strategy* aggregates those updates and returns the new global model, which then gets used in the next round of federated learning.

### Where is the accuracy?

You may have noticed that all metrics except for `losses_distributed` are empty. Where did the `{"accuracy: float(accuracy)"}` go?
Flower can automatically aggregate losses returned by individual clients, but it cannot do the same for metrics in the generic metrics dictionary. Metrics dictionary can contain very different kinds of metrics and even key/value pairs that are not metrics, so the framework does not know how to handle these automatically.
As users, we need to tell the framework how to handle/aggregate these custom metrics, and we do so by passing metric aggregation functions to the strategy. The strategy will then call these functions whenever it receives fit or evaluate metrics from clients. The two possible functions are `fit_metrics_aggregation_fn` and `evaluate_metrics_aggregation_fn`.
Let's create a simple weighted averaging function to aggregate the accuracy metric we return from evaluate:

In [16]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m['accuracy'] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

The only thing left to do is to tell the strategy to call this function whenever it receives evaluation metric dictionaries from the clients:

In [17]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=2,
    evaluate_metrics_aggregation_fn=weighted_average, # <--- pass the metric aggregation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-03-28 11:21:19,694	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7994344244.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3997172121.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

(ClientAppActor pid=6949) *** Classification Report ***:
(ClientAppActor pid=6949)               precision    recall  f1-score   support
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6949)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)     accuracy                           1.00        20
(ClientAppActor pid=6949)    macro avg       1.00      1.00      1.00        20
(ClientAppActor pid=6949) weighted avg       1.00      1.00      1.00        20
(ClientAppActor pid=6949) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)         

(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)         
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO

(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6949)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 


INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)         
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)         
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)         
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 0.79s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      		round 3: 0.0
INFO :      		round 4: 0.0
INFO :      		round 5: 0.0
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]}
INFO :      


(ClientAppActor pid=6949) 
(ClientAppActor pid=6949)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6949)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6949) 
(ClientAppActor pid=6949) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948)            0       1.00      1.00      1.00        12
(ClientAppActor pid=6948)            1       1.00      1.00      1.00         8
(ClientAppActor pid=6948) 
(ClientAppActor pid=6948) 


History (loss, distributed):
	round 1: 0.0
	round 2: 0.0
	round 3: 0.0
	round 4: 0.0
	round 5: 0.0
History (metrics, distributed, evaluate):
{'accuracy': [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]}

We now have a full system that performs federated training and federated evaluation. It uses the `weighted_average` function to aggregate custom evaluation metrics and calculate a single accuracy metric across all clients on the server side.

The other two categories of metrics (`losses_centralized` and `metrics_centralized`) are still empty because they only apply when centralized evaluation is being used.

## Final remarks

Congratulations, you just trained a Logistic Regression model, federated over 3 clients. With that, it was expected that you understand the basics of federated learning with Flower.